# A primer on Tensorflow

## As a general-purpose linear algebra compiler an and autodifferentiation toolkit

### Stefan Krastanov

In [ ]:
import numpy as np
import tensorflow as tf
import scipy
import scipy.linalg
import scipy.optimize

# Tensors, i.e. n-dim arrays

In [ ]:
array = np.random.rand(3,3,3)
array

In [ ]:
tf.convert_to_tensor(array)

# Our cost function in numpy and in tensorflow

In [ ]:
sigmaz_np = np.array([[1.,0.],[0.,-1.]])
sigmax_np = np.array([[0.,1.],[1., 0.]])
def h_np(p):
    return sigmaz_np + p*sigmax_np

def cost_np(p_array, v_in):
    v_out = v_in
    for p in p_array:
        v_out = scipy.linalg.expm(-1j*h_np(p)).dot(v_out)
    return -np.abs(v_out[0])

In [ ]:
sigmaz_tf = tf.constant(sigmaz_np, dtype=tf.dtypes.complex128)
sigmax_tf = tf.constant(sigmax_np, dtype=tf.dtypes.complex128)
def h_tf(p):
    return sigmaz_tf + tf.cast(p,tf.dtypes.complex128)*sigmax_tf

def cost_tf(p_array, v_in):
    v_out = v_in
    for p in p_array:
        v_out = tf.linalg.matvec(tf.linalg.expm(-1j*h_tf(p)), v_out)
    return -tf.abs(v_out[0])

In [ ]:
p_np = np.random.rand(10)
v_in_np = np.array([1,0])

p_tf = tf.constant(p_np)
v_in_tf = tf.constant(v_in_np, dtype=tf.dtypes.complex128)

# Timing them

In [ ]:
%timeit cost_np(p_np, v_in_np)

In [ ]:
%timeit cost_tf(p_tf, v_in_tf)

# So... Tensorflow is slower, function names are frequently different, support for complex numbers requires explicit casting... What is the point?

# Tensorflow is an optimizing compiler

In [ ]:
@tf.function
def h_tf(p):
    return sigmaz_tf + tf.cast(p,tf.dtypes.complex128)*sigmax_tf

@tf.function
def cost_tf(p_array, v_in):
    v_out = v_in
    for p in p_array:
        v_out = tf.linalg.matvec(tf.linalg.expm(-1j*h_tf(p)), v_out)
    return -tf.abs(v_out[0])

In [ ]:
# first time
%time cost_tf(p_tf, v_in_tf)

In [ ]:
# second time
%timeit cost_tf(p_tf, v_in_tf)

# Tensorflow is an autodiff framework

In [ ]:
p_np = np.random.rand(100)
p_tf = tf.constant(p_np)
variable = tf.Variable(p_tf)

In [ ]:
%%time
with tf.GradientTape() as tape:
    result = cost_tf(variable, v_in_tf)
    
tape.gradient(result, variable)

In [ ]:
%%time
scipy.optimize.approx_fprime(p_np, lambda v: cost_np(v, v_in_np), 0.0001)

# Tensorflow is a function optimization framework

In [ ]:
p_np = np.random.rand(100)
p_tf = tf.constant(p_np)
variable = tf.Variable(p_tf)

@tf.function
def h_tf(p):
    return sigmaz_tf + tf.cast(p,tf.dtypes.complex128)*sigmax_tf

@tf.function
def cost_tf(p_array, v_in):
    v_out = v_in
    for p in p_array:
        v_out = tf.linalg.matvec(tf.linalg.expm(-1j*h_tf(p)), v_out)
    return -tf.abs(v_out[0])

In [ ]:
optimizer = tf.optimizers.Nadam(learning_rate=0.01)

In [ ]:
cost_tf(variable, v_in_tf)

In [ ]:
with tf.GradientTape() as tape:
    result = cost_tf(variable, v_in_tf)
gradient = tape.gradient(result, variable)

optimizer.apply_gradients([(gradient, variable)])

In [ ]:
cost_tf(variable, v_in_tf)

But we do not want to write that much code every time... Tensorflow should know to take gradients if an optimizer is in use...

In [ ]:
@tf.function
def cost_with_implicit_arguments():
    return cost_tf(variable, v_in_tf)

In [ ]:
optimizer.minimize(cost_with_implicit_arguments, [variable])

In [ ]:
cost_tf(variable, v_in_tf)

In [ ]:
for i in range(50):
    print("\r Step %s"%i,end='',flush=True)
    optimizer.minimize(cost_with_implicit_arguments, [variable])

In [ ]:
cost_tf(variable, v_in_tf)

# An assortment of nice things and bad things about Tensorflow

## Matrix chain multiplication (disappointing)

In [ ]:
matrix = tf.constant(np.random.rand(600,600))
vector = tf.constant(np.random.rand(600,1))

In [ ]:
def naive():
    prod_matrix = matrix
    for i in range(10):
        prod_matrix = tf.matmul(matrix, prod_matrix)
    return tf.reduce_sum(tf.matmul(prod_matrix,vector))

In [ ]:
%time naive()

In [ ]:
def smart():
    prod_vec = tf.matmul(matrix,vector)
    for i in range(10):
        prod_vec = tf.matmul(matrix, prod_vec)
    return tf.reduce_sum(prod_vec)

In [ ]:
%time smart()

In [ ]:
@tf.function
def lazy():
    prod_matrix = matrix
    for i in range(10):
        prod_matrix = tf.matmul(matrix, prod_matrix)
    return tf.reduce_sum(tf.matmul(prod_matrix,vector))

In [ ]:
%time lazy()

In [ ]:
%time lazy()

## Einstein summation notation (convenient)

In [ ]:
matrix = tf.constant(np.random.rand(1000,1000))
vector = tf.constant(np.random.rand(1000,))

In [ ]:
tf.einsum("ij,j->i", matrix, vector)

In [ ]:
tf.einsum("i,ij,jk,k->", vector, matrix, matrix, vector)

Quiz time!

What does this do:

```
i,i->
ij,j->i
ij,i->j
i,i->i
ii->
ij,ij->ij
ijk,kl->ijl
i,ijk,kl->ijl
i->
```

In [ ]:
# some more tests of matrix chain multiplication performance, not discussed
@tf.function
def f():
    return tf.einsum("ij,jk,k->i", matrix, matrix, vector)
%time f()
%time f()

In [ ]:
# some more tests of matrix chain multiplication performance, not discussed
@tf.function
def f():
    return tf.linalg.matvec( tf.matmul(matrix,matrix), vector )
%time f()
%time f()

In [ ]:
# some more tests of matrix chain multiplication performance, not discussed
@tf.function
def f():
    return tf.linalg.matvec( matrix, tf.linalg.matvec(matrix, vector ) )
%time f()
%time f()

## Fast indices and slow indices (everyone has this problem)

In [111]:
matrix = tf.constant(np.random.rand(1000,1000))

In [116]:
%timeit tf.einsum("ij,jk->ik", matrix, matrix)

67.5 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
%timeit tf.einsum("ij,kj->ik", matrix, matrix)

70.2 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [118]:
%timeit tf.einsum("ji,kj->ik", matrix, matrix)

71.7 ms ± 846 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Multithreading (amazing)

In [ ]:
matrices = [tf.constant(np.random.normal(size=(200,200))) for i in range(10)]

In [ ]:
def f():
    new_matrices = []
    for m in matrices:
        new_m = m@m
        for nesting in range(10):
            new_m = new_m / tf.reduce_sum(new_m)
            new_m = new_m@new_m
        new_matrices.append(new_m)
    return sum(new_matrices)

In [ ]:
%time [f() for repetition in range(50)]

In [ ]:
@tf.function
def f():
    new_matrices = []
    for m in matrices:
        new_m = m@m
        for nesting in range(10):
            new_m = new_m / tf.reduce_sum(new_m)
            new_m = new_m@new_m
        new_matrices.append(new_m)
    return sum(new_matrices)

In [ ]:
f()
%time [f() for repetition in range(50)]

## Runs on GPUs and TPUs (amazing)

With a bit more code it can run on multiple GPUs, but that might require some profiling.

In [ ]:
with tf.device("GPU:0"):
    ...
    ...

# Plenty of other neat tools, but this is for today

# Alternatives to be aware of

- Theano - the first, it was lovely, not supported anymore
- PyTorch - the first to have "eager mode", but Tensorflow now also has that; poor support for complex numbers
- Julia - The Flux/Cassette/Zygote/DifferentialEquations libraries have immense potential, but are still somewhat experimental